In [ ]:
# Rutas fijas en Google Drive para cada dataset
DATASET_PATHS = {
    "aqua": "/content/drive/MyDrive/tesis/datasets/AQUA/test.json",
    "gsm8k": "/content/drive/MyDrive/tesis/datasets/GSM-8K/test.jsonl",
    "math_data": "/content/drive/MyDrive/tesis/datasets/MATH/math_data.jsonl",
    "math_shuffled": "/content/drive/MyDrive/tesis/datasets/MATH/shuffled_math.jsonl",
    "mmlu": "/content/drive/MyDrive/tesis/datasets/MMLU/MMLU_test.jsonl",
}


In [ ]:
import json
import os
import re
from typing import Dict, List, Optional
import sympy as sp
from sympy.parsing.latex import parse_latex as latex2sympy


import pandas as pd

def boxed_to_value_string(boxed_expr: str, precision: int = 15) -> str:
    """
    Convierte el contenido del \\boxed a string para tu dataset:
      - Si es numérico real, devuelve el valor como string flotante (%.{precision}g).
      - Si no es numérico (letra u otra expresión), devuelve la cadena tal cual.
      - Siempre retorna str.
    """
    s = (boxed_expr or "").strip()
    if not s:
        return ""
    try:
        sym = latex2sympy(s)
        # Intentar evaluar numéricamente con algo más de precisión
        val = sp.N(sym, precision + 5)
        # Si es real (o convertible a float), lo formateamos; si no, devolvemos la cadena original
        if val.is_real is False:
            return s
        # A veces is_real es None pero sí es convertible:
        f = float(val)
        return f"{f:.{precision}g}"
    except Exception:
        # No se pudo parsear / evaluar: devolver la cadena original
        return s

def extract_boxed_solution(latex: str) -> str:
    """
    Busca el primer \\boxed{...}, extrae lo que hay dentro.
    Si no hay, devuelve cadena vacía.
    """
    if latex is None:
        return ""
    s = str(latex)
    idx = s.find(r"\boxed{")
    if idx < 0:
        return ""
    start = idx + len(r"\boxed{")
    depth = 0
    i = start
    while i < len(s):
        if s[i] == "{":
            depth += 1
        elif s[i] == "}":
            if depth == 0:
                return s[start:i].strip()
            depth -= 1
        i += 1
    # si no encuentra cierre:
    return s[start:].strip()



class DatasetCSVBuilder:
    """
    Transforma datasets JSON/JSONL a CSV estandarizado para el pipeline experimental,
    con soporte para procesar solo los primeros `max_items` registros si se desea.
    """

    def __init__(self, dataset_paths: Dict[str, str]):
        self.dataset_paths = dataset_paths

        # Unidades comunes; ampliarlo si quieres
        self.unit_pattern = re.compile(r'[$€£%]|(?<![a-zA-Z])m\b|cm\b|kg\b|km\b')
        self.choice_pattern = re.compile(r'^\s*([A-Za-z])\)\s*(.*)$')
        self.boxed_pattern = re.compile(r'\\boxed\{([^}]*)\}')
        self.simple_tex_pattern = re.compile(r'\\text\{([^}]*)\}')

        # Columnas adicionales necesarias para el pipeline experimental completo
        self.experiment_columns = [
            "strategy",
            "model",
            "dataset",
            "resolvability_prompt",
            "resolvability_response",
            "is_octave_resolvable",
            "problem_features",
            "prompt",
            "model_output",
            "inference_time",
            "octave_code",
            "execution_output",
            "execution_error",
            "is_correct",
        ]

    # --------------------------------------------------------------------- #
    # API pública
    # --------------------------------------------------------------------- #
    def create_dataset(
        self,
        key: str,
        max_items: Optional[int] = None,
    ) -> pd.DataFrame:
        """
        Lee el dataset indicado por `key`, procesa hasta `max_items`
        registros (o todos si es None) y guarda el CSV resultante.
        """
        path = self.dataset_paths[key]
        records = self._load_records(path, max_items=max_items)
        df = pd.DataFrame(records)
        df = self._transform_dataframe(key, df)
        df = self._finalize_schema(df, dataset_name=key)
        self._save_as_csv(df, path, max_items=max_items)
        return df

    # --------------------------------------------------------------------- #
    # Funciones internas
    # --------------------------------------------------------------------- #
    def _load_records(
        self,
        path: str,
        *,
        max_items: Optional[int] = None,
    ) -> List[dict]:
        """
        Devuelve una lista de diccionarios cargados desde un .json o .jsonl.
        Si `max_items` está definido, corta la lista a ese tamaño.
        """
        data = []
        with open(path, "r", encoding="utf-8") as f:
            try:
                if path.endswith(".jsonl"):
                    for line in f:
                        if not line.strip():
                            continue
                        data.append(json.loads(line.strip()))
                        if max_items and len(data) >= max_items:
                            break
                else:  # .json (lista o objeto único)
                    content = json.load(f)
                    if isinstance(content, list):
                        data.extend(content[:max_items] if max_items else content)
                    else:
                        data.append(content)
            except json.JSONDecodeError as e:
                print(f"Error al leer {path}: {e}")

        return data

    def _transform_dataframe(self, name: str, df: pd.DataFrame) -> pd.DataFrame:
        """
        Normaliza columnas a un esquema común:
        - question, answer, rationale (strings)
        - otras columnas específicas por dataset (options, level, type, etc.)
        """
        df = df.copy()

        # Asegurar existencia de columnas base (se completan luego)
        for c in ["question", "answer", "rationale"]:
            if c not in df.columns:
                df[c] = None

        if name == "aqua":
            # AQUA-RAT: {question, options (list), correct (letra), rationale?}
            if "correct" in df.columns:
                df = df.rename(columns={"correct": "answer"})
            # Asegurar question
            if "question" not in df.columns and "Problem" in df.columns:
                df = df.rename(columns={"Problem": "question"})

            # Normalizar options a string ";"-separado
            if "options" in df.columns:
                df["options"] = df["options"].apply(
                    lambda x: ';'.join(x) if isinstance(x, list) else (x if isinstance(x, str) else "")
                )

            # Letra a mayúsculas y sin espacios
            df["answer"] = df["answer"].astype(str).str.strip().str.upper()

            # Mapear letra → texto de opción; limpiar unidades si hay
            df["answer_alternative"] = df.apply(self._map_if_unit, axis=1)

        elif name == "gsm8k":
            # GSM8K: {"question", "answer"} con "rationale #### final"
            if "question" not in df.columns and "Question" in df.columns:
                df = df.rename(columns={"Question": "question"})
            if "answer" in df.columns:
                parts = df["answer"].astype(str).str.split("####", n=1, expand=True)
                if parts.shape[1] == 2:
                    df["rationale"] = parts[0].str.strip()
                    df["answer"] = parts[1].str.strip()
                else:
                    # No hay '####': dejar answer tal cual y rationale vacío
                    df["rationale"] = ""
                    df["answer"] = parts[0].str.strip()

        elif name in ("math_data", "math_shuffled"):
          # 1) question := problem
          if "problem" in df.columns:
              df["question"] = df["problem"].astype(str)
          else:
              # si faltara 'problem', conserva 'question' si existe; si no, vacío
              df["question"] = df.get("question", "").astype(str)

          # 2) rationale := solution (solo para posibles estrategias que la usen)
          if "solution" in df.columns:
              df["rationale"] = df["solution"].astype(str)
          else:
              df["rationale"] = df.get("rationale", "").astype(str)

          # 3) answer := contenido de \boxed{...} convertido a string numérico si es evaluable; si no, tal cual
          df["answer"] = df["rationale"].apply(
              lambda s: extract_boxed_solution(s))


        elif name == "mmlu":
            # MMLU: nombres varían; soportar "question"/"Question", "choices"/"options"
            if "question" not in df.columns and "Question" in df.columns:
                df = df.rename(columns={"Question": "question"})
            if "choices" not in df.columns:
                # construir 'choices' desde columnas "Option*"
                option_cols = [c for c in df.columns if c.lower().startswith("option")]
                if option_cols:
                    df["choices"] = df[option_cols].apply(
                        lambda row: ';'.join(row.values.astype(str)), axis=1
                    )
            # answer suele venir ya como letra o texto; lo dejamos tal cual

        # Asegurar tipos string y strip básico
        for c in ["question", "answer", "rationale"]:
            df[c] = df[c].astype(str).fillna("").str.strip()

        # Agregar id incremental si no existe
        if "id" not in df.columns:
            df.insert(0, "id", range(len(df)))

        return df

    def _finalize_schema(self, df: pd.DataFrame, dataset_name: str) -> pd.DataFrame:
        # Añadir columnas experimentales si faltan
        for col in self.experiment_columns:
            if col not in df.columns:
                df[col] = dataset_name if col == "dataset" else None

        # Asegurar columnas mínimas
        for c in ["question", "answer", "rationale", "dataset"]:
            if c not in df.columns:
                df[c] = "" if c != "dataset" else dataset_name

        return df

    def _map_if_unit(self, row):
        """
        Para AQUA:
        - row['answer'] es una letra (A/B/C/...)
        - row['options'] es 'A) ...;B) ...;...'
        Devuelve el texto de opción mapeado; si incluye unidades, las quita.
        """
        letter = str(row.get("answer", "")).strip().upper()
        options = row.get("options", "")
        if not isinstance(options, str) or not options:
            return None

        # Construir mapping letra → texto
        mapping = {}
        for raw in [p for p in options.split(';') if p.strip()]:
            m = self.choice_pattern.match(raw)
            if m:
                k, v = m.group(1).upper(), m.group(2).strip()
                mapping[k] = v
            else:
                # Si no hay "A)" explícito, mapear por orden A,B,C,D...
                pass

        if not mapping:
            parts = [p.strip() for p in options.split(';') if p.strip()]
            abc = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
            mapping = {abc[i]: parts[i] for i in range(min(len(parts), len(abc)))}

        value = mapping.get(letter, None)
        if not value:
            return None

        # Quitar unidades comunes si aparecen pegadas
        cleaned = self.unit_pattern.sub('', value).strip()
        return cleaned if cleaned else value

    def _save_as_csv(
        self,
        df: pd.DataFrame,
        original_path: str,
        *,
        max_items: Optional[int] = None,
    ):
        """
        Guarda el CSV con un sufijo que indica si es un recorte parcial.
        """
        base_dir = os.path.dirname(original_path)
        base_name = os.path.splitext(os.path.basename(original_path))[0]
        suffix = f"_experiment_{max_items}" if max_items else "_experiment"
        output_path = os.path.join(base_dir, f"{base_name}{suffix}.csv")
        df.to_csv(output_path, index=False)
        print(f"✅ CSV experimental guardado en: {output_path}")


In [ ]:
# =========================
# BENCHMARK BUILDER (JSON)
# =========================
import os, json, random
import pandas as pd



BENCHMARK_OUT_JSON   = "/content/drive/MyDrive/tesis/datasets/benchmarks/benchmark_math_gsm8k_3x300.json"
BENCHMARK_OUT_JSONL  = "/content/drive/MyDrive/tesis/datasets/benchmarks/benchmark_math_gsm8k_3x300.jsonl"  # opcional
os.makedirs(os.path.dirname(BENCHMARK_OUT_JSON), exist_ok=True)

DATASETS   = ["gsm8k", "math_data", "math_shuffled"]
N_PER_DATASET = 300
SEED = 12345

def _build_processed_df(dataset_key: str) -> pd.DataFrame:
    """Procesa el dataset con tu builder y devuelve el DF normalizado."""
    builder = DatasetCSVBuilder(dataset_paths=DATASET_PATHS)
    df = builder.create_dataset(dataset_key)  # usa tus reglas (question, answer, rationale, dataset, etc.)
    # sanity: asegurar columnas mínimas
    for c in ("question", "answer", "rationale"):
        if c not in df.columns:
            df[c] = ""
    # filtra filas vacías sin pregunta
    df = df[df["question"].astype(str).str.strip() != ""].copy()
    df["dataset"] = dataset_key
    return df

def create_benchmark_json(
    datasets = DATASETS,
    n_per_dataset: int = N_PER_DATASET,
    seed: int = SEED,
    out_json_path: str = BENCHMARK_OUT_JSON,
    out_jsonl_path: str = None,   # p.ej. BENCHMARK_OUT_JSONL si lo quieres también en jsonl
):
    rng = random.Random(seed)
    all_records = []

    for key in datasets:
        print(f"→ Procesando dataset: {key}")
        df = _build_processed_df(key)

        # Si hay menos de n_per_dataset, toma todos
        take = min(n_per_dataset, len(df))
        sampled = df.sample(take, random_state=seed).copy()

        # Empaqueta solo los campos que quieres en el benchmark
        for idx, row in sampled.iterrows():
            rec = {
                "dataset": key,
                "id": f"{key}:{idx}",  # id estable combinando dataset e índice original
                "question":  str(row.get("question", "")),
                "answer":    str(row.get("answer", "")),
                "rationale": str(row.get("rationale", "")),
            }
            all_records.append(rec)

        print(f"  · seleccionados: {take} / {len(df)}")

    # Mezcla global para diversidad
    rng.shuffle(all_records)

    # Guarda como JSON (lista)
    with open(out_json_path, "w", encoding="utf-8") as f:
        json.dump(all_records, f, ensure_ascii=False, indent=2)
    print(f"\n✅ Benchmark JSON guardado en: {out_json_path}  (total={len(all_records)})")

    # (Opcional) JSONL: una línea por registro
    if out_jsonl_path:
        with open(out_jsonl_path, "w", encoding="utf-8") as f:
            for rec in all_records:
                f.write(json.dumps(rec, ensure_ascii=False) + "\n")
        print(f"✅ Benchmark JSONL guardado en: {out_jsonl_path}")

# === Ejecutar la construcción ===
create_benchmark_json(
    datasets=DATASETS,
    n_per_dataset=N_PER_DATASET,
    seed=SEED,
    out_json_path=BENCHMARK_OUT_JSON,
    out_jsonl_path=BENCHMARK_OUT_JSONL
)


→ Procesando dataset: gsm8k
✅ CSV experimental guardado en: /content/drive/MyDrive/tesis/datasets/GSM-8K/test_experiment.csv
  · seleccionados: 300 / 1319
→ Procesando dataset: math_data
✅ CSV experimental guardado en: /content/drive/MyDrive/tesis/datasets/MATH/math_data_experiment.csv
  · seleccionados: 300 / 5000
→ Procesando dataset: math_shuffled
✅ CSV experimental guardado en: /content/drive/MyDrive/tesis/datasets/MATH/shuffled_math_experiment.csv
  · seleccionados: 300 / 5000

✅ Benchmark JSON guardado en: /content/drive/MyDrive/tesis/datasets/benchmarks/benchmark_math_gsm8k_3x300.json  (total=900)
✅ Benchmark JSONL guardado en: /content/drive/MyDrive/tesis/datasets/benchmarks/benchmark_math_gsm8k_3x300.jsonl


In [ ]:
def show_benchmark(benchmark_path: str, n: int = 10, random_sample: bool = False):
    """
    Muestra por pantalla el benchmark creado.
    - benchmark_path: ruta al .json (lista) o .jsonl (1 objeto por línea)
    - n: cuántas filas mostrar
    - random_sample: si True, muestra muestra aleatoria; si False, muestra los primeros n
    """
    import os, json, pandas as pd
    from IPython.display import display

    if not os.path.exists(benchmark_path):
        raise FileNotFoundError(f"No existe el archivo: {benchmark_path}")

    # Cargar JSON (lista) o JSONL (línea por línea)
    records = []
    if benchmark_path.endswith(".jsonl"):
        with open(benchmark_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if line:
                    records.append(json.loads(line))
    else:
        with open(benchmark_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            if isinstance(data, list):
                records = data
            else:
                raise ValueError("El JSON no es una lista de objetos.")

    if not records:
        print("⚠️ El benchmark está vacío.")
        return

    df = pd.DataFrame.from_records(records)
    # Asegurar columnas esperadas
    for c in ("dataset", "id", "question", "answer", "rationale"):
        if c not in df.columns:
            df[c] = ""

    total = len(df)
    print(f"📦 Benchmark: {benchmark_path}")
    print(f"🔢 Total de ejercicios: {total}\n")

    # Distribución por dataset
    print("📊 Distribución por dataset:")
    display(df["dataset"].value_counts().rename_axis("dataset").to_frame("count"))

    # Vista rápida de ejemplos
    print(f"\n👀 {'Muestra aleatoria' if random_sample else 'Primeras filas'} (n={min(n, total)}):")
    view = df.sample(n, random_state=123) if random_sample and total > n else df.head(n)
    # Mostrar columnas principales primero
    cols = [c for c in ["dataset", "id", "question", "answer", "rationale"] if c in df.columns]
    display(view[cols])

    # Muestra 1 ejemplo completo (opcional, útil para inspección)
    print("\n🔎 Ejemplo completo (primer registro mostrado):")
    example = view.iloc[0].to_dict()
    print(json.dumps(example, ensure_ascii=False, indent=2))


In [ ]:
show_benchmark("/content/drive/MyDrive/tesis/datasets/benchmarks/benchmark_math_gsm8k_3x300.json", n=8, random_sample=True)

📦 Benchmark: /content/drive/MyDrive/tesis/datasets/benchmarks/benchmark_math_gsm8k_3x300.json
🔢 Total de ejercicios: 900

📊 Distribución por dataset:


,count
dataset,
math_shuffled,300
math_data,300
gsm8k,300



👀 Muestra aleatoria (n=8):


,dataset,id,question,answer,rationale
613,math_shuffled,math_shuffled:2807,How many ways are there to put 9 differently c...,20160,There are $9!$ ways to put the beads on the gr...
524,math_data,math_data:2769,The roots of the equation\n\[x^5 - 40x^4 + Px^...,32,Let the roots of the degree 5 polynomial be $\...
690,math_shuffled,math_shuffled:3474,There is a set of five positive integers whose...,7,"Since 5 is the median, there must be two integ..."
457,math_data,math_data:143,What is the largest possible value of $x$ such...,9,"Factoring gives us $(x - 9)(x + 4) = 0$, which..."
85,math_shuffled,math_shuffled:323,"The double factorial, denoted by $n!!$, return...",9,"For any $n$, we note that $n!!$ is the product..."
446,math_shuffled,math_shuffled:2205,What integer $n$ satisfies $0\le n<18$ and $$n...,9,An integer is divisible by $18$ if and only if...
294,gsm8k,gsm8k:55,Jean has 30 lollipops. Jean eats 2 of the loll...,14,Jean has 30 - 2 = <<30-2=28>>28 lollipops\nJea...
832,gsm8k,gsm8k:883,Elaina is holding the final concert in her tou...,25,"The concert, minus the encore, lasted for 65-m..."



🔎 Ejemplo completo (primer registro mostrado):
{
  "dataset": "math_shuffled",
  "id": "math_shuffled:2807",
  "question": "How many ways are there to put 9 differently colored beads on a $3\\times3$ grid if the purple bead and the green bead cannot be adjacent (either horizontally, vertically, or diagonally), and rotations and reflections of the grid are considered the same?",
  "answer": "20160",
  "rationale": "There are $9!$ ways to put the beads on the grid, not taking rotations, reflections, and the restriction on the purple and green beads into account. We need to subtract off the number of arrangements where the purple and green beads are adjacent from this number. There are $2\\cdot3=6$ horizontally adjacent pairs of locations, $3\\cdot2=6$ vertically adjacent ones, and $2\\cdot2+2\\cdot2=8$ pairs of diagonally adjacent locations. For each of these pairs, there are two ways to put the purple and green beads in them, and $7!$ ways to put the rest of the beads on the grid, givi

In [ ]:
show_benchmark("/content/drive/MyDrive/tesis/datasets/benchmarks/benchmark_math_gsm8k_3x300.jsonl", n=8, random_sample=False)

📦 Benchmark: /content/drive/MyDrive/tesis/datasets/benchmarks/benchmark_math_gsm8k_3x300.jsonl
🔢 Total de ejercicios: 900

📊 Distribución por dataset:


,count
dataset,
math_shuffled,300
math_data,300
gsm8k,300



👀 Primeras filas (n=8):


,dataset,id,question,answer,rationale
0,math_shuffled,math_shuffled:372,Determine the number of solutions in $x$ of th...,3,We can simplify the congruence as follows: \be...
1,math_data,math_data:4692,Simplify\n\[\cos^2 x + \cos^2 \left( \frac{\pi...,\frac{3}{2},"From the angle addition formula,\n\[\cos \left..."
2,math_data,math_data:345,Express the infinite series $$\frac{3}{206}+\f...,0.015,We start by factoring $\frac{1}{2}$ from all t...
3,math_data,math_data:4963,Let $\mathbf{a} = \begin{pmatrix} -2 \\ 5 \end...,\begin{pmatrix} 23/8 \\ 7/4 \end{pmatrix},The line containing $\mathbf{a}$ and $\mathbf{...
4,gsm8k,gsm8k:162,If a bag of marbles costs $20 and the price in...,92,When the price of the bag of marble increase b...
5,math_shuffled,math_shuffled:2826,Simplify the expression $$(x^5+3x^2+3x^5)-(x^7...,-x^7-2x^5+x^2,"Combining like terms, we find that \begin{ali..."
6,gsm8k,gsm8k:650,"On Tuesday, Clara bought 20 pomegranates at $2...",142,"On Tuesday, the pomegranates cost Clara 20 pom..."
7,gsm8k,gsm8k:300,"On Tuesday, Peter wants to exercise for twice ...",78,On Sunday and Monday he exercised a total of 3...



🔎 Ejemplo completo (primer registro mostrado):
{
  "dataset": "math_shuffled",
  "id": "math_shuffled:372",
  "question": "Determine the number of solutions in $x$ of the congruence $64x\\equiv 2\\pmod {66}$ such that $0< x\\le 100$.",
  "answer": "3",
  "rationale": "We can simplify the congruence as follows: \\begin{align*}\n64x&\\equiv 2\\pmod {66}\\\\\n32x&\\equiv 1\\pmod {33}\\\\\n-x&\\equiv 1\\pmod {33}\\\\\nx&\\equiv -1\\pmod{33}\\\\\nx&\\equiv 32\\pmod{33}.\n\\end{align*} The first few positive solutions to this are $32$, $32+33=65$, $32+2\\cdot 33=98$, after which the solutions are clearly greater than $100$ and so are extraneous. Thus there are $\\boxed{3}$ solutions in the given range."
}
